In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
from sklearn.base import clone
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.feature_selection import RFE, RFECV, SelectFromModel, SequentialFeatureSelector
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures

# Load data

In [3]:
X, y, feature_names, _, _ = load_boston().values()

In [103]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X, columns = feature_names), pd.Series(y, name='label'), test_size = .2, random_state=1)

In [104]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train: Linear Regression

In [98]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)

pred_lr = model_lr.predict(X_test)

print(model_lr.score(X_test, y_test))
print(mean_absolute_error(y_test, pred_lr))
print(mean_absolute_percentage_error(y_test, pred_lr))

0.7634174432138474
3.750712180838916
0.18776351124030624


In [24]:
pf = PolynomialFeatures(interaction_only=True, include_bias=False, degree=2)

X_train = pf.fit_transform(X_train)
X_test = pf.transform(X_test)

In [99]:
rfe = RFECV(estimator=model_lr, cv=5, scoring='r2')

X_train = rfe.fit_transform(X_train, y_train)
X_test = rfe.transform(X_test)

In [100]:
model_lr.fit(X_train, y_train)

pred_lr = model_lr.predict(X_test)

print(model_lr.score(X_test, y_test))
print(mean_absolute_error(y_test, pred_lr))
print(mean_absolute_percentage_error(y_test, pred_lr))

0.7635982049770087
3.7379107553033477
0.18737837457744916


# Train: Random Forest

In [105]:
X_train, X_test, y_train, y_test = train_test_split(pd.DataFrame(X, columns = feature_names), pd.Series(y, name='label'), test_size = .2, random_state=42)

In [106]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [107]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

model.score(X_test, y_test, )

0.8920995891343227

In [38]:
pf = PolynomialFeatures(interaction_only=True, include_bias=False, degree=2)

X_train = pf.fit_transform(X_train)
X_test = pf.transform(X_test)

In [39]:
rfe = RFECV(estimator=model, cv=5, scoring='r2')

X_train = rfe.fit_transform(X_train, y_train)
X_test = rfe.transform(X_test)

In [40]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

model.score(X_test, y_test, )

0.8556048645974019

## Recursive Feature Elimination

In [116]:
rfe = RFE(estimator=model, )

In [117]:
rfe.fit(X_train, y_train)

rfe.ranking_

array([1, 7, 5, 8, 2, 1, 3, 1, 6, 1, 1, 4, 1])

In [118]:
result = np.zeros(feature_names.size)
rk = np.argsort(model.feature_importances_)
ch = int(feature_names.size/2)
result[rk[-ch:]] = 1

result[rk[:-ch]] = np.arange(feature_names.size - ch +1, 1, -1)

In [119]:
np.alltrue(result == rfe.ranking_)

True

## Sequencial Feature Selection: forward

In [95]:
sfs = SequentialFeatureSelector(estimator=model, cv=3)

In [97]:
sfs.fit(X_train, y_train)

sfs.support_

array([False, False, False, False,  True,  True, False, False,  True,
        True,  True, False,  True])

In [120]:
features_support = np.zeros(X_train.shape[1], dtype = bool)

In [121]:
n_select = X_train.shape[1]//2
n_cv =3

In [122]:
for _ in range(n_select):
    non_selected = np.flatnonzero(~features_support)
    dict_score = dict()
    for f in non_selected:
        candidates = features_support.copy()
        candidates[f] = True
        cvs = cross_val_score(estimator=model, X= X_train[:, candidates], y= y_train, cv=n_cv).mean()
        dict_score[f] = cvs
    selected = max(dict_score, key=lambda x: dict_score[x])
    features_support[selected] = True

In [125]:
np.alltrue(features_support==sfs.support_)

True

## Select from model

In [55]:
sfm = SelectFromModel(estimator=model, threshold='mean')

In [56]:
sfm.fit(X_train, y_train)

sfm.get_support()

array([False, False, False, False, False,  True, False, False, False,
       False, False, False,  True])

In [25]:
sfm.threshold_

0.07692307692307693

In [28]:
model.feature_importances_.mean()

0.07692307692307693

In [51]:
class SFM:
    def __init__(self, estimator, strategy='mean'):
        self.estimator = clone(estimator)
        self.strategy = strategy
    
    def fit(self, X, y):
        self.estimator.fit(X, y)
        
    def get_support(self):
        try:
            self.importance = self.estimator.feature_importances_
        except:
            self.importance = np.abs(self.estimator.coef_.flatten())
            
        if self.strategy == 'mean':
            self.threshold = self.importance.mean()
        elif self.strategy == 'median':
            self.threshold = np.median(self.importance)
        else:
            self.threshold = self.strategy.copy()
            
        return self.importance > self.threshold

In [52]:
sfm0 = SFM(estimator=model, strategy='mean')

In [53]:
sfm0.fit(X_train, y_train)

sfm0.get_support()

array([False, False, False, False, False,  True, False, False, False,
       False, False, False,  True])

In [57]:
np.alltrue(sfm0.get_support() == sfm.get_support())

True